Advanced Chart Customization

In [10]:
# importing libraries
import ast
import pandas as pd
from datasets import load_dataset
import matplotlib.pyplot as plt

# loading datasets
dataset = load_dataset('lukebarousse/data_jobs')
df = dataset['train'].to_pandas()

# cleaning dataset
df['job_posted_date'] = pd.to_datetime(df['job_posted_date'])
df['job_skills'] = (df['job_skills']
                    .apply(lambda x: ast.literal_eval(x) 
                           if pd.notna(x) else x))

#filter for Data Analyst Data
df_DS = df[df['job_title_short'] == 'Data Science'].copy()

df_DS

,job_title_short,job_title,job_location,job_via,job_schedule_type,job_work_from_home,search_location,job_posted_date,job_no_degree_mention,job_health_insurance,job_country,salary_rate,salary_year_avg,salary_hour_avg,company_name,job_skills,job_type_skills


In [9]:
df_DS['job_posted_month_num'] = df_DS['job_posted_date'].dt.month

df_DS_explode = df_DS.explode('job_skills')

df_DS_pivot = df_DS_explode.pivot_table(index = 'job_posted_month_num', columns= 'job_skills', aggfunc = 'size', fill_value = 0)

df_DS_pivot


job_skills
job_posted_month_num


In [ ]:
df_DS_pivot.loc['Total'] = df_DS_pivot.sum()

# df_DS_pivot[df_DS_pivot.loc['Total'].sort_values(ascending=False).head().index]

df_DS_pivot = (df_DS_pivot[df_DS_pivot.loc['Total']
                           .sort_values(ascending=False)
                           .head().index])

df_DS_pivot = df_DS_pivot.drop('Total')

df_DS_pivot = df_DS_pivot.reset_index()

df_DS_pivot['job_posted_month'] = (df_DS_pivot['job_posted_month_num']
                                   .apply(lambda x: 
                                          pd.to_datetime(x, format ='%m')
                                          .strftime('%b')))

df_DS_pivot = df_DS_pivot.set_index('job_posted_month')

df_DS_pivot = df_DS_pivot.drop(columns='job_posted_month_num')

df_DS_pivot.plot(kind='line')

plt.title('Top 5 Skills for Data Scientists Monthly')
plt.ylabel('Count')
plt.xlabel('Month')
plt.show()